In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.python.data import AUTOTUNE

from aegomoku.game_data import read_training_data

In [2]:
directory = "/Users/wgiersche/workspace/Project-Ellie/DATA/"
filenames = [os.path.join(directory, filename) for filename in os.listdir(directory)]
print(filenames)
BOARD_SIZE=15

['/Users/wgiersche/workspace/Project-Ellie/DATA/00011.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00003.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00001.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00009.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00005.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00007.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00010.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00002.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00012.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00008.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00004.pickle', '/Users/wgiersche/workspace/Project-Ellie/DATA/00006.pickle']


In [3]:
filename = filenames[0]
print(f"Reading from {filename}")
examples = read_training_data(filename, BOARD_SIZE)
len(examples), len(examples[0]), examples[0][0].shape
#examples = examples[:5]

Reading from /Users/wgiersche/workspace/Project-Ellie/DATA/00011.pickle


(6032, 3, (17, 17, 3))

In [4]:
def serialize_example(state, policy, value):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  feature = {
      's': tf.train.Feature(bytes_list=tf.train.BytesList(value=[state])),
      'p': tf.train.Feature(bytes_list=tf.train.BytesList(value=[policy])),
      'v': tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
  }

  # Create a Features message using tf.train.Example.

  e = tf.train.Example(features=tf.train.Features(feature=feature))
  return e.SerializeToString()

In [5]:
outfile = 'test2.tfrecord'

In [6]:
# Write the `tf.train.Example` observations to the file.
with tf.io.TFRecordWriter(outfile) as writer:
    for state, policy, value in examples:
        state = tf.io.serialize_tensor(state.astype(np.uint8)).numpy()
        policy = tf.io.serialize_tensor(np.array(np.array(policy)*255).astype(np.uint8)).numpy()
        example = serialize_example(state, policy, value)
        writer.write(example)

2022-09-04 21:43:56.285671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
s, p, v = examples[0]

In [8]:
len(tf.io.serialize_tensor(s.astype(np.uint8)).numpy())

886

In [9]:
len(tf.io.serialize_tensor(np.array(p).astype(np.uint8)).numpy())

237

In [10]:
filenames = [outfile]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [11]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  #print(example)

In [12]:
# Create a description of the features.
feature_description = {
    's': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'p': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'v': tf.io.FixedLenFeature([], tf.float32, default_value=0),
}

dictionary =tf.io.parse_single_example(raw_record, feature_description)
dictionary['p'].numpy()

b'\x08\x04\x12\x05\x12\x03\x08\xe1\x01"\xe1\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00<B\x00\x00C<\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [13]:
def read_tfrecord(example):

    record = tf.io.parse_single_example(example, feature_description)
    s = tf.io.parse_tensor(record['s'], out_type=tf.uint8)
    p = tf.io.parse_tensor(record['p'], out_type=tf.uint8)
    v = record['v']
    return s, p, v

In [14]:
def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [15]:
def get_dataset(filenames, batch_size):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(batch_size)
    return dataset

In [16]:
ds = get_dataset(outfile, 8)

In [17]:
s, p, v = next(iter(ds))

In [18]:
s

<tf.Tensor: shape=(8, 17, 17, 3), dtype=uint8, numpy=
array([[[[0, 0, 1],
         [0, 0, 1],
         [0, 0, 1],
         ...,
         [0, 0, 1],
         [0, 0, 1],
         [0, 0, 1]],

        [[0, 0, 1],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 1]],

        [[0, 0, 1],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 1]],

        ...,

        [[0, 0, 1],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 1]],

        [[0, 0, 1],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 1]],

        [[0, 0, 1],
         [0, 0, 1],
         [0, 0, 1],
         ...,
         [0, 0, 1],
         [0, 0, 1],
         [0, 0, 1]]],


       [[[0, 0, 1],
         [0, 0, 1],
         [0, 0, 1],
         ...,
         [0, 0, 1],
         [0, 0, 1],
